<a href="https://colab.research.google.com/github/yuuuchen/ProgressPal/blob/main/prompt_0815.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/yuuuchen/ProgressPal.git
%cd ProgressPal

Prompt 模板庫（Template Library）

In [ ]:
# 全域模板庫
'''
simple：(精簡)一句話指令，直接用三個欄位填入
structured：(分段式)角色設定 + 背景 + 限制
dialogue:(對話式)模擬師生對話
example_driven：(範例驅動)提供回答範例

# 要加入 請以自然語言輸出，避免使用 Markdown、項目符號或表格，直接用完整句子表達。
# 請用 5 句話以內完成回答。

'''

PROMPT_TEMPLATES = {
    "simple": """你是一位{tone}的資料結構助教。
請用{style}方式，面對一位感到{emotion}的學生，
回答以下問題：{question}
請根據以下教材回答，避免超出範圍：
{materials}
""",

    "structured": """[角色設定]
你是一位{tone}的資料結構助教。

[任務]
面對一位感到{emotion}的學生，請用{style}方式，
協助解答以下問題，並確保依據教材內容。

[學生問題]
{question}

[教材]
{materials}

[回答限制]
1. 不要超出教材內容
2. 使用貼合情緒的語氣
""",

    "dialogue": """以下是你與一位學生的對話：
學生（感到{emotion}）: {question}
助教（{tone}，採用{style}）:
請根據以下教材回應學生：
{materials}
""",

    "example_driven": """你是一位{tone}的資料結構助教，擅長用{style}方式解釋。
請回答感到{emotion}的學生的問題：{question}
請根據以下教材回答，並至少提供一個具體例子：
{materials}
"""
}

CURRENT_PROMPT_MODE = "simple"

In [ ]:
# 映射方法：情緒 → 語氣 + 教學策略
def map_emotion_to_profile(emotion):
  mapping = {
      "frustrated": {"tone": "溫暖且安撫", "style": "循序漸進、拆解問題"},
      "confused": {"tone": "溫和且耐心", "style": "舉例對照、比喻解釋"},
      "bored": {"tone": "活潑且有趣", "style": "加入情境化案例、互動提問"},
      "engaged": {"tone": "積極且肯定", "style": "深入探討、引導延伸思考"},
      "surprised": {"tone": "熱情且鼓勵", "style": "延伸趣味點、引入新視角"},
      "joyful": {"tone": "輕鬆且正向", "style": "融入挑戰題、鼓勵自我探索"},
  }
  return mapping.get(emotion, {"tone": "中性", "style": "一般解釋"})

# 主方法：Prompt 生成方法
def generate_prompt(emotion, question, materials):
  learner_profile = map_emotion_to_profile(emotion)  # 情緒解析層

  materials_text = "\n".join(f"{i+1}. {m}" for i, m in enumerate(materials))

  # 使用模板組合 Prompt
  template = PROMPT_TEMPLATES[CURRENT_PROMPT_MODE]
  prompt_text = template.format(
      tone=learner_profile["tone"],
      style=learner_profile["style"],
      emotion=emotion,
      question=question,
      materials=materials_text
  )

  output_format_hint = "以自然語言分段回答，語氣溫暖、易於理解，最後總結困惑點，避免使用 Markdown 或表格，限制在150字以內"
  return f"{prompt_text.strip()}\n\n請注意輸出格式：{output_format_hint}"

使用範例：

In [ ]:
test_input = {
  "emotion": "confused",
  "question": "linked list 和 array 差在哪裡？",
  "materials": [
      "Linked List 是由節點組成，每個節點包含資料與指向下一個節點的指標。",
      "Array 的記憶體分配是連續的，存取速度快，但插入與刪除成本高。"
  ]
}
result = generate_prompt(
    emotion=test_input["emotion"],
    question=test_input["question"],
    materials=test_input["materials"]
)

print("=== Prompt Text ===")
print(result)

=== Prompt Text ===
你是一位溫和且耐心的資料結構助教。
請用舉例對照、比喻解釋方式，面對一位感到confused的學生，
回答以下問題：linked list 和 array 差在哪裡？
請根據以下教材回答，避免超出範圍：
1. Linked List 是由節點組成，每個節點包含資料與指向下一個節點的指標。
2. Array 的記憶體分配是連續的，存取速度快，但插入與刪除成本高。

請注意輸出格式：以自然語言分段回答，語氣溫暖、易於理解，最後總結困惑點，避免使用 Markdown 或表格，限制在150字以內


找最佳prompt結構

In [ ]:
test_input = {
  "emotion": "confused",
  "question": "linked list 和 array 差在哪裡？",
  "materials": [
      "Linked List 是由節點組成，每個節點包含資料與指向下一個節點的指標。",
      "Array 的記憶體分配是連續的，存取速度快，但插入與刪除成本高。"
  ]
}
for i in PROMPT_TEMPLATES.keys():
  CURRENT_PROMPT_MODE = i
  result = generate_prompt(
      emotion=test_input["emotion"],
      question=test_input["question"],
      materials=test_input["materials"]
  )
  print(f"=== Mode：{i} ===")
  print(result)
  print("\n")

=== Mode：simple ===
你是一位溫和且耐心的資料結構助教。
請用舉例對照、比喻解釋方式，面對一位感到confused的學生，
回答以下問題：linked list 和 array 差在哪裡？
請根據以下教材回答，避免超出範圍：
1. Linked List 是由節點組成，每個節點包含資料與指向下一個節點的指標。
2. Array 的記憶體分配是連續的，存取速度快，但插入與刪除成本高。

請注意輸出格式：以自然語言分段回答，語氣溫暖、易於理解，最後總結困惑點，避免使用 Markdown 或表格，限制在150字以內


=== Mode：structured ===
[角色設定]
你是一位溫和且耐心的資料結構助教。

[任務]
面對一位感到confused的學生，請用舉例對照、比喻解釋方式，
協助解答以下問題，並確保依據教材內容。

[學生問題]
linked list 和 array 差在哪裡？

[教材]
1. Linked List 是由節點組成，每個節點包含資料與指向下一個節點的指標。
2. Array 的記憶體分配是連續的，存取速度快，但插入與刪除成本高。

[回答限制]
1. 不要超出教材內容
2. 使用貼合情緒的語氣

請注意輸出格式：以自然語言分段回答，語氣溫暖、易於理解，最後總結困惑點，避免使用 Markdown 或表格，限制在150字以內


=== Mode：dialogue ===
以下是你與一位學生的對話：
學生（感到confused）: linked list 和 array 差在哪裡？
助教（溫和且耐心，採用舉例對照、比喻解釋）:
請根據以下教材回應學生：
1. Linked List 是由節點組成，每個節點包含資料與指向下一個節點的指標。
2. Array 的記憶體分配是連續的，存取速度快，但插入與刪除成本高。

請注意輸出格式：以自然語言分段回答，語氣溫暖、易於理解，最後總結困惑點，避免使用 Markdown 或表格，限制在150字以內


=== Mode：example_driven ===
你是一位溫和且耐心的資料結構助教，擅長用舉例對照、比喻解釋方式解釋。
請回答感到confused的學生的問題：linked list 和 array 差在哪裡？
請根據以下教材回答，並至少提供一個具體例子：
1. Linked 